In [1]:
!pip install xlrd
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [2]:
df = pd.read_excel('GSAF5.xls')

In [3]:
df.head()

Date    Year        Type    Country              State  \
0  04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
1  02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
2  25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
3  14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   
4  04-Feb-2024  2024.0    Provoked   TRINIDAD                NaN   

                           Location      Activity                 Name Sex  \
0                Old Man's, Waikiki       Surfing        Matthew White   M   
1                    Rainbows, Oahu      Swimming                  NaN   F   
2        Sandlnd Island, Jurian Bay           NaN               female   F   
3  Vaitarna River, Palghar District       Fishing  Vicky Suresh Govari   M   
4                               NaN  Spearfishing                 male   M   

   Age                          Injury Unnamed: 11   Time  \
0  NaN  No injury, shark bit surfboard           N    NaN   
1   11        Lacerations to left foot           N  13h30   
2   46                      Leg bitten           N  11h30   
3   32  Calf of lower left leg injured           N    NaN   
4  NaN                 Shoulder bitten           N  06h30   

              Species                         Source  pdf href formula href  \
0       Tiger shark 8'             Surfer, 3/6/2024F  NaN          NaN  NaN   
1       3' to 4' shark     Hawaii News Now, 3/4/2024  NaN          NaN  NaN   
2          Tiger shark           WA Today, 2/26/2024  NaN          NaN  NaN   
3       Bull shark, 7'     Times of India, 2/14/2024  NaN          NaN  NaN   
4  Blacktip reef shark  Trinidad Guardian, 2/11/2014  NaN          NaN  NaN   

  Case Number Case Number.1  original order Unnamed: 21 Unnamed: 22  
0         NaN           NaN             NaN         NaN         NaN  
1         NaN           NaN             NaN         NaN         NaN  
2         NaN           NaN             NaN         NaN         NaN  
3         NaN           NaN             NaN         NaN         NaN  
4         NaN           NaN             NaN         NaN         NaN

In [4]:
df.shape

(6968, 23)

**Activity, Sex, Injury, Time, Species**

**Sex**

In [5]:
df["Sex"].unique()

array(['M', 'F', nan, ' M', 'M ', 'lli', 'M x 2', 'N', '.'], dtype=object)

In [6]:
def normalise_sex(df):
    sex_mapping = {
    'M': 'M',
    'F': 'F', 
    ' M': 'M',
    'M ': 'M',
    'lli': 'M',
    'M x 2': 'M',
    '.': 'U',
    }

    df["Sex"] = df["Sex"].map(sex_mapping)
    df["Sex"] = df["Sex"].map(sex_mapping).fillna('U')
    return df

In [7]:
normalise_sex(df)

Date    Year        Type    Country              State  \
0     04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
1     02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
2     25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
3     14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   
4     04-Feb-2024  2024.0    Provoked   TRINIDAD                NaN   
...           ...     ...         ...        ...                ...   
6963          NaN     NaN         NaN        NaN                NaN   
6964          NaN     NaN         NaN        NaN                NaN   
6965          NaN     NaN         NaN        NaN                NaN   
6966          NaN     NaN         NaN        NaN                NaN   
6967          NaN     NaN         NaN        NaN                NaN   

                              Location      Activity                 Name Sex  \
0                   Old Man's, Waikiki       Surfing        Matthew White   M   
1                       Rainbows, Oahu      Swimming                  NaN   F   
2           Sandlnd Island, Jurian Bay           NaN               female   F   
3     Vaitarna River, Palghar District       Fishing  Vicky Suresh Govari   M   
4                                  NaN  Spearfishing                 male   M   
...                                ...           ...                  ...  ..   
6963                               NaN           NaN                  NaN   U   
6964                               NaN           NaN                  NaN   U   
6965                               NaN           NaN                  NaN   U   
6966                               NaN           NaN                  NaN   U   
6967                               NaN           NaN                  NaN   U   

      Age                          Injury Unnamed: 11   Time  \
0     NaN  No injury, shark bit surfboard           N    NaN   
1      11        Lacerations to left foot           N  13h30   
2      46                      Leg bitten           N  11h30   
3      32  Calf of lower left leg injured           N    NaN   
4     NaN                 Shoulder bitten           N  06h30   
...   ...                             ...         ...    ...   
6963  NaN                             NaN         NaN    NaN   
6964  NaN                             NaN         NaN    NaN   
6965  NaN                             NaN         NaN    NaN   
6966  NaN                             NaN         NaN    NaN   
6967  NaN                             NaN         NaN    NaN   

                 Species                         Source  pdf  \
0          Tiger shark 8'             Surfer, 3/6/2024F  NaN   
1          3' to 4' shark     Hawaii News Now, 3/4/2024  NaN   
2             Tiger shark           WA Today, 2/26/2024  NaN   
3          Bull shark, 7'     Times of India, 2/14/2024  NaN   
4     Blacktip reef shark  Trinidad Guardian, 2/11/2014  NaN   
...                   ...                           ...  ...   
6963                  NaN                           NaN  NaN   
6964                  NaN                           NaN  NaN   
6965                  NaN                           NaN  NaN   
6966                  NaN                           NaN  NaN   
6967                  NaN                           NaN  NaN   

                                           href formula href Case Number  \
0                                                   NaN  NaN         NaN   
1                                                   NaN  NaN         NaN   
2                                                   NaN  NaN         NaN   
3                                                   NaN  NaN         NaN   
4                                                   NaN  NaN         NaN   
...                                                 ...  ...         ...   
6963  http://sharkattackfile.net/spreadsheets/pdf_di...  NaN         NaN   
6964  http://sharkattackfile.net/spreadsh

**Activity**

In [8]:
df.dtypes

Date               object
Year              float64
Type               object
Country            object
State              object
Location           object
Activity           object
Name               object
Sex                object
Age                object
Injury             object
Unnamed: 11        object
Time               object
Species            object
Source             object
pdf                object
href formula       object
href               object
Case Number        object
Case Number.1      object
original order    float64
Unnamed: 21        object
Unnamed: 22        object
dtype: object

In [9]:
df["Activity"] = df["Activity"].fillna('Undisclosed')

In [10]:
activities = []

for i in df["Activity"]:
    lst = i.split()
    
    if len(lst) == 1:
        if lst[0].isalpha():          
            activities.append(lst[0])
        
activities = set(activities)

In [11]:
activities

{'Angling',
 'Bather',
 'Bathing',
 'Batin',
 'Boat',
 'Boating',
 'Bodyboarding',
 'Bodysurfing',
 'Canoeing',
 'Clamming',
 'Crabbing',
 'Crawling',
 'Crayfishing',
 'Cruising',
 'Diving',
 'Fihing',
 'Filming',
 'Fisherman',
 'Fishing',
 'Fishingat',
 'Floating',
 'Foilboarding',
 'Freediving',
 'Hilo',
 'Jumping',
 'Kakaying',
 'Kayaking',
 'Kiteboarding',
 'Kitesurfing',
 'Lobstering',
 'Murder',
 'Murdered',
 'Oystering',
 'Paddleboarding',
 'Paddleskiing',
 'Paddling',
 'Parasailing',
 'Playing',
 'Rescuing',
 'Rowing',
 'SUP',
 'Sailing',
 'Scalloping',
 'Sculling',
 'Shipwreck',
 'Shipwrecked',
 'Shrimping',
 'Sightseeing',
 'Sitting',
 'Skimboarding',
 'Skindiving',
 'Snorkeling',
 'Spearfishing',
 'Spearishing',
 'Splashing',
 'Stamding',
 'Standing',
 'Suicide',
 'Surfing',
 'Surfng',
 'Swimming',
 'Swimmingq',
 'Swmming',
 'Undisclosed',
 'Unknown',
 'Wading',
 'Wakeboarding',
 'Walking',
 'Washing',
 'Watercraft',
 'Windsurfing',
 'canoeing',
 'male'}

In [12]:
#Breakdown activities by category
unknown_activities = {
    'Murder',
    'Suicide',
    'UN',
    'Unknown',
    'male',
    'Hilo',
    'Filming',
    'Rescuing',
    'Shipwreck',
    'Shipwrecked',
    'Sightseeing'
}

diving_activites = {
    'Skindiving',
    'Snorkeling',
    'Diving'
}

boating_activities = {
    'Boat',
    'Boating',
    'Canoeing',
    'Kayaking',
    'Paddleboarding',
    'Paddleskiing',
    'Rowing',
    'Sailing',
    'Wakeboarding',
    'Watercraft',
    'Parasailing'
}

swimming_and_bathing_activities = {
    'Bather',
    'Bathing',
    'Sitting',
    'Splashing',
    'Stamding',
    'Standing',
    'Washing',
    'Swimming',
    'Swimmingq',
    'Swmming',
    'Playing',
    'Jumping'
}

surfing_activities = {
    'Bodyboarding',
    'Bodysurfing',
    'Foilboarding',
    'Kiteboarding',
    'Kitesurfing',
    'Skimboarding',
    'Surfing',
    'Windsurfing'
}

fishing_activities = {
    'Angling',
    'Crabbing',
    'Crawling',
    'Crayfishing',
    'Fishing',
    'Fishingat',
    'Lobstering',
    'Oystering',
    'Spearfishing',
    'Spearishing',
    'Shrimping',
    'Batin'
}

In [13]:
def get_activity(activity):
    if activity == 'Undisclosed':
        return 'Undisclosed'

    activity = activity.lower()

    for i in diving_activites:
        if fuzz.partial_ratio(i.lower(), activity) > 75:
            return 'Diving'
            
    for i in boating_activities:
        if fuzz.partial_ratio(i.lower(), activity) > 75:
            return 'Boating'

    for i in swimming_and_bathing_activities:
        if fuzz.partial_ratio(i.lower(), activity) > 75:
            return 'Swimming/Bathing'

    for i in surfing_activities:
        if fuzz.partial_ratio(i.lower(), activity) > 75:
            return 'Surfing'

    for i in fishing_activities:
        if fuzz.partial_ratio(i.lower(), activity) > 75:
            return 'Fishing'
    
    return 'Not categorised'

In [14]:
df["Activity"].value_counts()

Activity
Surfing                                                        1121
Swimming                                                        975
Undisclosed                                                     611
Fishing                                                         488
Spearfishing                                                    382
                                                               ... 
Fishing boat swamped in a storm                                   1
Free diving (but on surface)                                      1
Adrift after the sinking of the motor yacht Princess Dianne       1
Free diving for abalone (submerged)                               1
Wreck of  large double sailing canoe                              1
Name: count, Length: 1588, dtype: int64

In [15]:
df["Activity"] = df["Activity"].apply(get_activity)

In [16]:
df.head()

Date    Year        Type    Country              State  \
0  04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
1  02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
2  25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
3  14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   
4  04-Feb-2024  2024.0    Provoked   TRINIDAD                NaN   

                           Location          Activity                 Name  \
0                Old Man's, Waikiki           Surfing        Matthew White   
1                    Rainbows, Oahu  Swimming/Bathing                  NaN   
2        Sandlnd Island, Jurian Bay       Undisclosed               female   
3  Vaitarna River, Palghar District           Fishing  Vicky Suresh Govari   
4                               NaN           Fishing                 male   

  Sex  Age                          Injury Unnamed: 11   Time  \
0   M  NaN  No injury, shark bit surfboard           N    NaN   
1   F   11        Lacerations to left foot           N  13h30   
2   F   46                      Leg bitten           N  11h30   
3   M   32  Calf of lower left leg injured           N    NaN   
4   M  NaN                 Shoulder bitten           N  06h30   

              Species                         Source  pdf href formula href  \
0       Tiger shark 8'             Surfer, 3/6/2024F  NaN          NaN  NaN   
1       3' to 4' shark     Hawaii News Now, 3/4/2024  NaN          NaN  NaN   
2          Tiger shark           WA Today, 2/26/2024  NaN          NaN  NaN   
3       Bull shark, 7'     Times of India, 2/14/2024  NaN          NaN  NaN   
4  Blacktip reef shark  Trinidad Guardian, 2/11/2014  NaN          NaN  NaN   

  Case Number Case Number.1  original order Unnamed: 21 Unnamed: 22  
0         NaN           NaN             NaN         NaN         NaN  
1         NaN           NaN             NaN         NaN         NaN  
2         NaN           NaN             NaN         NaN         NaN  
3         NaN           NaN             NaN         NaN         NaN  
4         NaN           NaN             NaN         NaN         NaN

In [17]:
df["Activity"].value_counts()

Activity
Surfing             1348
Swimming/Bathing    1330
Fishing             1207
Diving              1127
Not categorised      700
Boating              645
Undisclosed          611
Name: count, dtype: int64

**Species**

In [18]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Unnamed: 11', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')

In [19]:
df["Species "].value_counts().sum()

3811

In [20]:
species_list = []

df["Species "] = df["Species "].fillna('Unknown')

for i in df["Species "]:
    
    if 'shark' in i.lower():
        split = i.lower().split()
        index = 0
        for i in split:
            if 'shark' in i:
                break
            else:
                index += 1
        if split[index -1].isalpha():
            species_list.append(split[index - 1])

In [21]:
species_list

['tiger',
 'tiger',
 'bull',
 'reef',
 'bull',
 'white',
 'raggedtooth',
 'white',
 'white',
 'sevengill',
 'confirmed',
 'confirmed',
 'bull',
 'lemon',
 'white',
 'whitetip',
 'white',
 'tiger',
 'nurse',
 'tiger',
 'galapagos',
 'tiger',
 'cookiecutter',
 'white',
 'blue',
 'blue',
 'wfite',
 'wobbegong',
 'whitetip',
 'tiger',
 'tiger',
 'tiger',
 'rreef',
 'reef',
 'reef',
 'confirmes',
 'white',
 'bull',
 'tiger',
 'bull',
 'bull',
 'whaler',
 'tiger',
 'reef',
 'lemon',
 'bull',
 'bull',
 'small',
 'bull',
 'mako',
 'nurse',
 'tiger',
 'tiger',
 'white',
 'tiger',
 'nurse',
 'bull',
 'white',
 'white',
 'bull',
 'white',
 'blacktip',
 'white',
 'bull',
 'blue',
 'nurse',
 'blacktip',
 'confirmed',
 'lemon',
 'small',
 'sandtiger',
 'whitetip',
 'whitetip',
 'confirmed',
 'while',
 'tiger',
 'white',
 'whaler',
 'bull',
 'blacktip',
 'tiger',
 'epaulette',
 'tiger',
 'bull',
 'blacktip',
 'white',
 'tiger',
 'nurse',
 'bull',
 'white',
 'wobbegong',
 'wobbegong',
 'sevengill',
 '

In [22]:
species_list = [
    ("Tiger Shark", ["tiger", "tiger shark", "tigerr"]),
    ("Bull Shark", ["bull"]),
    ("Blacktip Shark", ["blacktip", "blacktip shark", "bluacktip"]),
    ("White Shark", ["white", "whale"]),
    ("Raggedtooth Shark", ["raggedtooth", "sandtiger"]),
    ("Sevengill Shark", ["sevengill"]),
    ("Lemon Shark", ["lemon"]),
    ("Oceanic Whitetip Shark", ["oceanic", "oceanic whitetip", "oceaniic"]),
    ("Nurse Shark", ["nurse"]),
    ("Cookiecutter Shark", ["cookiecutter"]),
    ("Blue Shark", ["blue"]),
    ("Wobbegong Shark", ["wobbegong"]),
    ("Caribbean Reef Shark", ["caribbean", "reef"]),
    ("Grey Nurse Shark", ["grey"]),
    ("Bronze Whaler Shark", ["bronze", "bronz"]),
    ("Mako Shark", ["mako"]),
    ("Spinner Shark", ["spinner"]),
    ("Galapagos Shark", ["galapagos"]),
    ("Tope Shark", ["tope"]),
    ("Epaulette Shark", ["epaulette"]),
    ("Angel Shark", ["angel"]),
    ("Reef Shark", ["reef"]),
    ("Silky Shark", ["silky"]),
    ("Thresher Shark", ["thresher"]),
    ("Hammerhead Shark", ["hammerhead"]),
    ("Shovelnose Shark", ["shovelnose"]),
    ("Tawny Shark", ["tawny"]),
    ("Dogfish Shark", ["dogfish"])
]

In [23]:
def get_species(x):
    x = x.lower()
    for species, variations in species_list:
        species = species.lower()
        for variation in variations:
            if fuzz.partial_ratio(variation.lower(), x) > 70:
                return species.capitalize()
    return 'Other'

In [24]:
pattern = r'(\d+(?:\.\d+)?)\s*m'

df["Shark Size"] = df["Species "].str.extract(pattern) + 'm'

df["Shark Size"].fillna('Unknown', inplace=True)

In [25]:
df.head()

Date    Year        Type    Country              State  \
0  04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
1  02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
2  25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
3  14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   
4  04-Feb-2024  2024.0    Provoked   TRINIDAD                NaN   

                           Location          Activity                 Name  \
0                Old Man's, Waikiki           Surfing        Matthew White   
1                    Rainbows, Oahu  Swimming/Bathing                  NaN   
2        Sandlnd Island, Jurian Bay       Undisclosed               female   
3  Vaitarna River, Palghar District           Fishing  Vicky Suresh Govari   
4                               NaN           Fishing                 male   

  Sex  Age                          Injury Unnamed: 11   Time  \
0   M  NaN  No injury, shark bit surfboard           N    NaN   
1   F   11        Lacerations to left foot           N  13h30   
2   F   46                      Leg bitten           N  11h30   
3   M   32  Calf of lower left leg injured           N    NaN   
4   M  NaN                 Shoulder bitten           N  06h30   

              Species                         Source  pdf href formula href  \
0       Tiger shark 8'             Surfer, 3/6/2024F  NaN          NaN  NaN   
1       3' to 4' shark     Hawaii News Now, 3/4/2024  NaN          NaN  NaN   
2          Tiger shark           WA Today, 2/26/2024  NaN          NaN  NaN   
3       Bull shark, 7'     Times of India, 2/14/2024  NaN          NaN  NaN   
4  Blacktip reef shark  Trinidad Guardian, 2/11/2014  NaN          NaN  NaN   

  Case Number Case Number.1  original order Unnamed: 21 Unnamed: 22 Shark Size  
0         NaN           NaN             NaN         NaN         NaN    Unknown  
1         NaN           NaN             NaN         NaN         NaN    Unknown  
2         NaN           NaN             NaN         NaN         NaN    Unknown  
3         NaN           NaN             NaN         NaN         NaN    Unknown  
4         NaN           NaN             NaN         NaN         NaN    Unknown

In [26]:
df["Shark Size"].value_counts()

Shark Size
Unknown    5583
3m          173
1.8m        138
1.5m        126
2m          123
           ... 
1.96m         1
1.65m         1
2.27m         1
3.1m          1
4.7m          1
Name: count, Length: 62, dtype: int64

In [27]:
df["Species "] = df["Species "].apply(get_species)

In [28]:
df["Shark Size"] = df["Species "].apply(lambda x: get_species(x)[1])

In [29]:
def classify_size(length):
    try:
        length = float(length)
        if length > 3.0:
            return "Large"
        elif 1.5 <= length <= 3.0:
            return "Medium"
        else:
            return "Small"
    except ValueError:
        return "Unknown"

df["Shark Size"] = df["Shark Size"].apply(classify_size)

In [30]:
df["Shark Size"].value_counts()

Shark Size
Unknown    6968
Name: count, dtype: int64

In [31]:
df["Species "].value_counts()

Species 
Other                     4475
Tiger shark               1585
Bull shark                 216
Blue shark                 143
Nurse shark                112
Mako shark                  58
Caribbean reef shark        58
Wobbegong shark             55
Lemon shark                 50
Raggedtooth shark           44
Hammerhead shark            43
White shark                 30
Blacktip shark              26
Sevengill shark             19
Silky shark                 13
Dogfish shark               10
Galapagos shark              9
Grey nurse shark             7
Bronze whaler shark          6
Shovelnose shark             4
Oceanic whitetip shark       2
Angel shark                  2
Tope shark                   1
Name: count, dtype: int64

**WHY**

In [32]:
#####df["Shark Size"] = df["Shark Size"].fillna(df["Species "].apply(lambda x: get_species(x)[-1])

**Time**

In [33]:
df["Time"] = df["Time"].fillna('Unknown')

In [34]:
def get_time(x):
    if x == 'Unknown':
        return x
    
    try:
        x = str(x)  
        x = x.split('h')[0]
        x = int(x)
        
        if x > 23:                 
            x = int(x / 100)
        
        time_ranges = {
            'Morning': (6, 11),    # '06:00', '11:59'
            'Afternoon': (12, 17), # '12:00', '17:59'
            'Evening': (18, 21),   # '18:00', '21:59'
        }
     
        for t_range, (start, end) in time_ranges.items():
            if start <= x <= end:
                return t_range
        
        if (22 <= x <= 24) | (0 <= x <= 5):
            return 'Night'
        
        return None

    except:
        time_ranges = ['Morning', 'Afternoon', 'Evening', 'Night']

        for t_range in time_ranges:
            if fuzz.partial_ratio(t_range, x) > 70:
                return t_range
           
        return None
        
df["Time"] = df["Time"].apply(get_time)

In [35]:
df["Time"].value_counts()

Time
Unknown      3551
Afternoon    1782
Morning      1088
Evening       290
Night         128
Name: count, dtype: int64

In [36]:
df["Time"].unique()

array(['Unknown', 'Afternoon', 'Morning', 'Evening', 'Night', None],
      dtype=object)

In [37]:
none_count = (df["Time"] == None).sum()
print("Number of occurrences of 'None' in the 'Time' column:", none_count)

Number of occurrences of 'None' in the 'Time' column: 0


**Injury**

In [38]:
df.head()

Date    Year        Type    Country              State  \
0  04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
1  02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
2  25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
3  14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   
4  04-Feb-2024  2024.0    Provoked   TRINIDAD                NaN   

                           Location          Activity                 Name  \
0                Old Man's, Waikiki           Surfing        Matthew White   
1                    Rainbows, Oahu  Swimming/Bathing                  NaN   
2        Sandlnd Island, Jurian Bay       Undisclosed               female   
3  Vaitarna River, Palghar District           Fishing  Vicky Suresh Govari   
4                               NaN           Fishing                 male   

  Sex  Age                          Injury Unnamed: 11       Time  \
0   M  NaN  No injury, shark bit surfboard           N    Unknown   
1   F   11        Lacerations to left foot           N  Afternoon   
2   F   46                      Leg bitten           N    Morning   
3   M   32  Calf of lower left leg injured           N    Unknown   
4   M  NaN                 Shoulder bitten           N    Morning   

         Species                         Source  pdf href formula href  \
0     Tiger shark             Surfer, 3/6/2024F  NaN          NaN  NaN   
1           Other     Hawaii News Now, 3/4/2024  NaN          NaN  NaN   
2     Tiger shark           WA Today, 2/26/2024  NaN          NaN  NaN   
3      Bull shark     Times of India, 2/14/2024  NaN          NaN  NaN   
4  Blacktip shark  Trinidad Guardian, 2/11/2014  NaN          NaN  NaN   

  Case Number Case Number.1  original order Unnamed: 21 Unnamed: 22 Shark Size  
0         NaN           NaN             NaN         NaN         NaN    Unknown  
1         NaN           NaN             NaN         NaN         NaN    Unknown  
2         NaN           NaN             NaN         NaN         NaN    Unknown  
3         NaN           NaN             NaN         NaN         NaN    Unknown  
4         NaN           NaN             NaN         NaN         NaN    Unknown

In [39]:
df["Injury"].unique()

array(['No injury, shark bit surfboard', 'Lacerations to left foot',
       'Leg bitten', ..., 'FATAL, leg stripped of flesh  ',
       'FATAL, knocked overboard by tail of shark & carried off by shark ',
       'FATAL. "Shark bit him in half, carrying away the lower extremities" '],
      dtype=object)

In [41]:
df["Injury"].value_counts()

Injury
FATAL                                                                              863
Foot bitten                                                                        100
Survived                                                                            97
No injury                                                                           85
Leg bitten                                                                          81
                                                                                  ... 
Right calf & heel bitten                                                             1
Multiple bites to foot after jumping off surfboard onto shark PROVOKED INCIDENT      1
Shallow  lacerations to left  thigh PROVOKED INCIDENT                                1
Bitten on finger                                                                     1
FATAL. "Shark bit him in half, carrying away the lower extremities"                  1
Name: count, Length: 4078, dtype: in

In [44]:
def normalize_injury(df): 
    df["Injury"] = df["Injury"].apply(lambda x: "Fatal" if "fatal" in x.lower() else x)
    df["Injury"] = df["Injury"].apply(lambda x: "Severe" if "bitten" in x.lower() else x)
    return df

In [45]:
df["Injury"].value_counts()

Injury
FATAL                                                                              863
Foot bitten                                                                        100
Survived                                                                            97
No injury                                                                           85
Leg bitten                                                                          81
                                                                                  ... 
Right calf & heel bitten                                                             1
Multiple bites to foot after jumping off surfboard onto shark PROVOKED INCIDENT      1
Shallow  lacerations to left  thigh PROVOKED INCIDENT                                1
Bitten on finger                                                                     1
FATAL. "Shark bit him in half, carrying away the lower extremities"                  1
Name: count, Length: 4078, dtype: in

In [ ]:
df["Injury"].unique()